In [1]:
#download ok for 2014 & 2015
#2016 data needs curation

In [1]:
#!conda install -y -c conda-forge gcsfs pyarrow
!pip install gcsfs pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.2/110.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.0/218.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 26.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1
  Attempting uninstall: fssp

In [2]:
import pandas as pd

import dask
import dask.dataframe as dd
from dask.distributed import Client, progress, wait

from gcsfs import GCSFileSystem


In [3]:
gcs = GCSFileSystem(token='anon')
gcs.ls('anaconda-public-data/nyc-taxi/csv/2016')

['anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-01.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-02.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-03.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-04.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-05.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-06.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-07.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-08.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-09.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-10.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-11.csv',
 'anaconda-public-data/nyc-taxi/csv/2016/yellow_tripdata_2016-12.csv']

In [14]:
with gcs.open('anaconda-public-data/nyc-taxi/csv/2014/yellow_tripdata_2014-01.csv') as f:
    df = pd.read_csv(f, nrows=5)                     
    #df = pd.read_csv(f, nrows=5, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

df.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2014-01-09 20:45:25,2014-01-09 20:52:31,1,0.7,-73.994770,40.736828,1,N,-73.982227,40.731790,CRD,6.5,0.5,0.5,1.40,0,8.90
1,CMT,2014-01-09 20:46:12,2014-01-09 20:55:12,1,1.4,-73.982392,40.773382,1,N,-73.960449,40.763995,CRD,8.5,0.5,0.5,1.90,0,11.40
2,CMT,2014-01-09 20:44:47,2014-01-09 20:59:46,2,2.3,-73.988570,40.739406,1,N,-73.986626,40.765217,CRD,11.5,0.5,0.5,1.50,0,14.00
3,CMT,2014-01-09 20:44:57,2014-01-09 20:51:40,1,1.7,-73.960213,40.770464,1,N,-73.979863,40.777050,CRD,7.5,0.5,0.5,1.70,0,10.20
4,CMT,2014-01-09 20:47:09,2014-01-09 20:53:32,1,0.9,-73.995371,40.717248,1,N,-73.984367,40.720524,CRD,6.0,0.5,0.5,1.75,0,8.75


In [5]:
def pandas_to_parquet(year):
    for i in range(1, 13):
        fname = "yellow_tripdata_%d-%02d" % (year, i)
        fname_csv = "%s.csv" % fname
        fname_parquet = "%s.parquet" % fname

        print("downloading %s" % fname_csv)
        with gcs.open('anaconda-public-data/nyc-taxi/csv/%d/%s' % (year,fname_csv)) as f:
            df = pd.read_csv(f, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

        #print("converting to parquet: %s" % fname_parquet)
        #df.to_parquet("data/%s" % fname_parquet)
    
        print("Writing local csv: %s" % fname_csv)
        df.to_csv("data/%s" % fname_csv)

#pandas_to_parquet(2015)        

Parallelize Pandas with Dask.dataframe

In [6]:
client = Client()
client

2022-12-25 21:54:03,774 - distributed.diskutils - INFO - Found stale lock file and directory '/rapids/notebooks/host/docker-pyspark/dask-docker/workspace/dask-worker-space/worker-gti63lgh', purging
2022-12-25 21:54:03,775 - distributed.diskutils - INFO - Found stale lock file and directory '/rapids/notebooks/host/docker-pyspark/dask-docker/workspace/dask-worker-space/worker-u791f7j7', purging
2022-12-25 21:54:03,775 - distributed.diskutils - INFO - Found stale lock file and directory '/rapids/notebooks/host/docker-pyspark/dask-docker/workspace/dask-worker-space/worker-67rgrohd', purging
2022-12-25 21:54:03,775 - distributed.diskutils - INFO - Found stale lock file and directory '/rapids/notebooks/host/docker-pyspark/dask-docker/workspace/dask-worker-space/worker-oynz03s3', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 31.36 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43017,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 31.36 GiB
Comm: tcp://127.0.0.1:39587,Total threads: 2
Dashboard: http://127.0.0.1:43607/status,Memory: 7.84 GiB
Nanny: tcp://127.0.0.1:39599,


In [16]:
nyc_datatype = {'VendorID': 'string',
                'passenger_count': 'int32',
                'trip_distance': 'float32',
                'pickup_longitude': 'float32',
                'pickup_latitude': 'float32',
                'RateCodeID': 'string',
                'store_and_fwd_flag': 'string',
                'dropoff_longitude': 'float32',
                'dropoff_latitude': 'float32',
                'payment_type': 'string',
                'fare_amount': 'float32',
                'extra': 'float32',
                'mta_tax': 'float32',
                'tip_amount': 'float32',
                'tolls_amount': 'float32',
                'improvement_surcharge': 'float32',
                'total_amount':'float32' }

nyc_datatype2014 = {'vendor_id': 'string',
                ' passenger_count': 'int32',
                ' trip_distance': 'float32',
                ' pickup_longitude': 'float32',
                ' pickup_latitude': 'float32',
                ' rate_code': 'string',
                ' store_and_fwd_flag': 'string',
                ' dropoff_longitude': 'float32',
                ' dropoff_latitude': 'float32',
                ' payment_type': 'string',
                ' fare_amount': 'float32',
                #'extra': 'float32',
                ' mta_tax': 'float32',
                ' tip_amount': 'float32',
                ' tolls_amount': 'float32',
                ' surcharge': 'float32',
                ' total_amount':'float32' }

#for c in df.columns:
#    print("<%s>" % c)

In [11]:
def data_2014_adjustment(df):
    #2014 data colums renaming
    colnames = []
    for c in df.columns:
        colnames += [c.replace(' ', '')]

    df.columns = colnames
    df = df.rename(columns={'pickup_datetime':'tpep_pickup_datetime',
                       'dropoff_datetime':'tpep_dropoff_datetime',
                       'rate_code':'RateCodeID',
                       'vendor_id':'VendorID',
                       'surcharge':'improvement_surcharge' })
    df["extra"] = 0

    df = df[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
           'passenger_count', 'trip_distance', 'pickup_longitude',
           'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
           'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
           'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
           'improvement_surcharge', 'total_amount']]
    #print(df.columns)
    return df

#2014 Data with columns adjustments

In [17]:
def dask_to_parquet(year):
    for i in range(1, 13):
        fname = "yellow_tripdata_%d-%02d" % (year, i)
        fname_csv = "%s.csv" % fname
        fname_parquet = "%s.parquet" % fname

        print("downloading %s" % fname_csv)
        df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/%d/%s' % (year, fname_csv),
                         storage_options={'token': 'anon'}, 
                         dtype=nyc_datatype2014,
                         parse_dates=[' pickup_datetime', ' dropoff_datetime'])
            
        print("writing local parquet: %s" % fname_parquet)
        df = data_2014_adjustment(df)
        df = df.astype(nyc_datatype)

        df = df.persist()
        wait(df)
        df.to_parquet('data/nyc-taxi/parquet/%s' % fname_parquet, overwrite=True)
                
dask_to_parquet(2014)

downloading yellow_tripdata_2014-01.csv
writing local parquet: yellow_tripdata_2014-01.parquet
downloading yellow_tripdata_2014-02.csv
writing local parquet: yellow_tripdata_2014-02.parquet
downloading yellow_tripdata_2014-03.csv
writing local parquet: yellow_tripdata_2014-03.parquet
downloading yellow_tripdata_2014-04.csv
writing local parquet: yellow_tripdata_2014-04.parquet
downloading yellow_tripdata_2014-05.csv
writing local parquet: yellow_tripdata_2014-05.parquet
downloading yellow_tripdata_2014-06.csv
writing local parquet: yellow_tripdata_2014-06.parquet
downloading yellow_tripdata_2014-07.csv
writing local parquet: yellow_tripdata_2014-07.parquet
downloading yellow_tripdata_2014-08.csv
writing local parquet: yellow_tripdata_2014-08.parquet
downloading yellow_tripdata_2014-09.csv
writing local parquet: yellow_tripdata_2014-09.parquet
downloading yellow_tripdata_2014-10.csv
writing local parquet: yellow_tripdata_2014-10.parquet
downloading yellow_tripdata_2014-11.csv
writing lo

In [16]:
df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2014/yellow_*.csv',
                 storage_options={'token': 'anon'}, 
                 dtype={' tolls_amount': 'float64'},
                 parse_dates=[' pickup_datetime', ' dropoff_datetime'])
#.head(n=5)

In [17]:
df = data_2014_adjustment(df)
df = df.astype(nyc_datatype)

df = df.persist()
progress(df)
#wait(df)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')


VBox()

In [21]:
#%time print(df.passenger_count.sum().compute())

#df.to_parquet('data/yellow_tripdata_2014.parquet', overwrite=True)

#2015 Data

In [ ]:
def dask_to_parquet(year):
    for i in range(1, 13):
        fname = "yellow_tripdata_%d-%02d" % (year, i)
        fname_csv = "%s.csv" % fname
        fname_parquet = "%s.parquet" % fname

        print("downloading %s" % fname_csv)
        df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/%d/%s' % (year, fname_csv),
                         storage_options={'token': 'anon'}, 
                         dtype = nyc_datatype,
                         parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
            
        print("writing local parquet: %s" % fname_parquet)
        df = df.persist()
        wait(df)
        df.to_parquet('data/nyc-taxi/parquet/%s' % fname_parquet, overwrite=True)
        
        #pandas_df = df.compute()
        #pandas_df.head()
        #pandas_df.to_csv("data/%s" % fname_csv)
        
dask_to_parquet(2015)

In [16]:
df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2015/yellow_*.csv',
                 storage_options={'token': 'anon'}, 
                 parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])


In [17]:
df = df.persist()
progress(df)
#wait(df)

VBox()

In [18]:
%time print(df.passenger_count.sum().compute())
#df = df.astype(nyc_datatype)

df.to_parquet('data/yellow_tripdata_2015.parquet', overwrite=True)

245566747
CPU times: user 29.9 ms, sys: 1.34 ms, total: 31.3 ms
Wall time: 226 ms


#2016 Data

Data cleansing required, lot of errors

worker2_1    | Exception: 'ValueError(\'Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.\\n\\n+---------------+--------+----------+\\n
| Column        | Found  | Expected |\\n
+---------------+--------+----------+\\n| VendorID      | object | int64    |\\n| trip_distance | object | float64  |\\n+---------------+--------+----------+\\n\\nThe following columns also raised exceptions on conversion:\\n\\n- VendorID\\n  ValueError("invalid literal for int() with base 10: \\\'2016-11-01 00:17:01\\\'")\\n- trip_distance\\n  ValueError("could not convert string to float: \\\'N\\\'")\\n\\nUsually this is due to dask\\\'s dtype inference failing, and\\n*may* be fixed by specifying dtypes manually by adding:\\n\\ndtype={\\\'VendorID\\\': \\\'object\\\',\\n       \\\'trip_distance\\\': \\\'object\\\'}\\n\\nto the call to `read_csv`/`read_table`.\\n\\n-------------------------------------------------------------\\n\\nThe following columns also failed to properly parse as dates:\\n\\n- tpep_pickup_datetime\\n- tpep_dropoff_datetime\\n\\nThis is usually due to an invalid value in that column. To\\ndiagnose and fix it\\\'s recommended to drop these columns from the\\n`parse_dates` keyword, and manually convert them to dates later\\nusing `dd.to_datetime`.\')'

In [47]:
df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2016/yellow_*.csv',
                 storage_options={'token': 'anon'}, 
                 dtype={'VendorID': 'int32',
                        'passenger_count': 'int32',
                        'trip_distance': 'string',
                        'pickup_longitude': 'float32',
                        'pickup_latitude': 'float32',
                        'RateCodeID': 'int64',
                        'store_and_fwd_flag': 'string',
                        'dropoff_longitude': 'float32',
                        'dropoff_latitude': 'float32',
                        'payment_type': 'float32',
                        'fare_amount': 'float32',
                        'extra': 'float32',
                        'mta_tax': 'float32',
                        'tip_amount': 'string',
                        'tolls_amount': 'float32',
                        'improvement_surcharge': 'float32',
                        'total_amount':'float32' },
                 parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [48]:
df = df.persist()
progress(df)
#wait(df)

VBox()

In [18]:
df.dtypes

VendorID                         string
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int32
trip_distance                   float32
pickup_longitude                float32
pickup_latitude                 float32
RateCodeID                       string
store_and_fwd_flag               string
dropoff_longitude               float32
dropoff_latitude                float32
payment_type                     string
fare_amount                     float32
extra                           float32
mta_tax                         float32
tip_amount                      float32
tolls_amount                    float32
improvement_surcharge           float32
total_amount                    float32
dtype: object

In [32]:
#%time print(df.passenger_count.sum().compute())
print(df.compute())

#df.to_parquet('data/yellow_tripdata_2016.parquet', overwrite=True)

                                  VendorID tpep_pickup_datetime  \
2 2016-12-31 15:15:01  2016-12-31 15:15:09                    1   
1 2016-12-01 00:00:01  2016-12-01 00:10:22                    1   
  2016-12-01 00:00:01  2016-12-01 00:11:01                    1   
2 2016-12-01 00:00:02  2016-12-01 00:09:17                    6   
  2016-12-01 00:00:02  2016-12-01 00:15:20                    1   
...                                    ...                  ...   
1 2016-12-31 23:59:54  2017-01-01 00:19:41                    1   
2 2016-12-31 23:59:54  2017-01-01 00:04:29                    1   
  2016-12-31 23:59:54  2017-01-01 00:09:26                    2   
  2016-12-31 23:59:54  2017-01-01 00:09:44                    1   
1 2016-12-31 23:59:58  2017-01-01 00:03:50                    1   

                      tpep_dropoff_datetime  passenger_count trip_distance  \
2 2016-12-31 15:15:01                   0.0                1             N   
1 2016-12-01 00:00:01                  